In [1]:
import keras
import tensorflow as tf
print('TensorFlow version:', tf.__version__) 
print('Keras version:', keras.__version__)
import sys
import os
from os.path import join
import json
import random
import itertools
import re
import datetime
#import cairocffi as cairo
import editdistance
import numpy as np
from scipy import ndimage
import pylab
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model, load_model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks
import cv2


/home/user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


TensorFlow version: 1.10.1
Keras version: 2.2.2


In [2]:
sess = tf.Session()
K.set_session(sess)
from keras.models import model_from_json

DEVICE = "/cpu:0"  # /use cpu:0 for cpu only or /gpu:0

In [3]:
working_dir = os.getcwd()
print("Working directory is:",working_dir)
sys.path.append(working_dir)  # To find local version of the library

ROOT_DIR = os.path.join("/home/user/Source/Evrim/Turkishplate") #linux
#ROOT_DIR = os.path.join("C:/Users/is95217/source/plate/turkishplate") #win
sys.path.append(ROOT_DIR + "/Python")


DATA_DIR = "/home/user/Source/data/dataEvrim/Original_15Kcrop_distributed"  #linux
#DATA_DIR = "C:/Users/is95217/data/Original_15Kcrop_distributed" #win
print("Data:", DATA_DIR)
print("Source code:", ROOT_DIR)

Working directory is: /home/user/Source/Evrim/Turkishplate
Data: /home/user/Source/data/dataEvrim/Original_15Kcrop_distributed
Source code: /home/user/Source/Evrim/Turkishplate


In [4]:
sys.path.append(os.path.dirname(os.path.realpath('PPlate')) + "/python")
import PPlate as p

# LOAD MODEL 

In [5]:
# load json and create model
#fmodel='lr0.02_mom0.9'
#fmodel = '/Model/Plate15K'

#loaded_model=p.load_model(ROOT_DIR + '/Model/plate15K')
loaded_model=p.load_model(ROOT_DIR + '/Model/model15k_2GRU')
 



# Test on validation images

In [19]:
cnt_test= len(os.listdir(DATA_DIR + '/test'))

tiger_test = p.TextImageGenerator(DATA_DIR, '/test', 128, 64, cnt_test, 4) # batch size 
tiger_test.build_data()

net_inp = loaded_model.get_layer(name='the_input').input
net_out = loaded_model.get_layer(name='softmax').output

In [7]:
"""

cnt_test= 2000#len(os.listdir(DATA_DIR + '/train'))

tiger_test = p.TextImageGenerator(DATA_DIR, '/train', 128, 64, cnt_test, 4) # batch size 
tiger_test.build_data()

net_inp = loaded_model.get_layer(name='the_input').input
net_out = loaded_model.get_layer(name='softmax').output
"""

"\n\ncnt_test= 2000#len(os.listdir(DATA_DIR + '/train'))\n\ntiger_test = p.TextImageGenerator(DATA_DIR, '/train', 128, 64, cnt_test, 4) # batch size \ntiger_test.build_data()\n\nnet_inp = loaded_model.get_layer(name='the_input').input\nnet_out = loaded_model.get_layer(name='softmax').output\n"

In [20]:
tiger_test.batch_size,tiger_test.img_h,tiger_test.img_w

(1536, 64, 128)

#  Full test set

In [21]:
for inp_value, _ in tiger_test.next_batch():
    matched=0
    non_matched=0
    testset_cnt = inp_value['the_input'].shape[0]
    X_data = inp_value['the_input']
    
    net_out_value = sess.run(net_out, feed_dict={net_inp:X_data})
    pred_texts = p.decode_batch(net_out_value)
    
    labels = inp_value['the_labels']
    texts = []     
    
    for label in labels:
        text = p.labels_to_text(label)        
        texts.append(text)       
    
    for cnt in range(testset_cnt):        
        
        if pred_texts[cnt] == texts[cnt]:
            matched += 1            
        else:
            non_matched +=  1
            if len(pred_texts[cnt])<7:
                print('wrong Predicted: %s True: %s i: %s len_pred: %s len_true: %s' % (pred_texts[cnt], texts[cnt],str(cnt),
                                                                              len(pred_texts[cnt]),len(texts[cnt])))
            
    print("matched=",matched, ",total=",testset_cnt , "TP: %", round(matched/testset_cnt, 2) * 100)
    break
    

wrong Predicted: 3EB501 True: 43EB501 i: 492 len_pred: 6 len_true: 7
wrong Predicted: 43PH79 True: 43PH797 i: 521 len_pred: 6 len_true: 7
wrong Predicted: 58L165 True: 58VL165 i: 845 len_pred: 6 len_true: 7
wrong Predicted: 411469 True: 41B1769 i: 1125 len_pred: 6 len_true: 7
wrong Predicted: 41F221 True: 41F2216 i: 1241 len_pred: 6 len_true: 7
wrong Predicted: 5MHR20 True: 35MHR20 i: 1310 len_pred: 6 len_true: 7
wrong Predicted: 4KL293 True: 14KL293 i: 1334 len_pred: 6 len_true: 7
wrong Predicted: 41V601 True: 41VE601 i: 1431 len_pred: 6 len_true: 7
wrong Predicted: 41T56 True: 41BD346 i: 1505 len_pred: 5 len_true: 7
matched= 1376 ,total= 1536 TP: % 90.0


In [10]:
#matched= 1071 ,total= 1568 TP: % 68.0  25 epoch 15k  test accuracy
#matched= 1082 ,total= 1567 TP: % 69.0 epoch 15k  val accuracy
#matched= 1380 ,total= 2000 TP: % 69.0 train accuracy


